In [23]:
import re
import warnings

import numpy as np
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from gensim.models import FastText
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('../data/Indonesian Sentiment Twitter Dataset Labeled.csv', sep="\t")
df.head()

In [5]:
alay_dict = pd.read_csv("../data/new_kamusalay.csv", encoding = 'latin-1', header=None)
alay_dict = alay_dict.rename(columns=  {0: 'original',
                                        1: 'replacement'})
stopword_dict = pd.read_csv('../data/stopwordbahasa.csv', header=None)
stopword_dict = stopword_dict.rename(columns={0: 'stopword'})

In [6]:
df.isnull().sum()

In [7]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#lowercase
def lowercase(text):
    return text.lower()

def remove_unnecessary_char(text):
    text = re.sub('\n',' ',text) # Remove every '\n'
    text = re.sub('rt',' ',text) # Remove every retweet symbol
    text = re.sub('((@[^\s]+)|(#[^\s]+))',' ',text)
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text) # Remove every URL
    text = re.sub('  +', ' ', text) # Remove extra spaces
    return text
    
def remove_nonaplhanumeric(text):
    text = re.sub('[^0-9a-zA-Z]+', ' ', text) 
    return text

alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))
def normalize_alay(text):
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])

def remove_stopword(text):
    text = ' '.join(['' if word in stopword_dict.stopword.values else word for word in text.split(' ')])
    text = re.sub('  +', ' ', text) # Remove extra spaces
    text = text.strip()
    return text

def stemming(text):
    return stemmer.stem(text)

def preprocess(text):
    text = lowercase(text) # 1
    text = remove_unnecessary_char(text) # 2
    text = remove_nonaplhanumeric(text) # 3
    text = normalize_alay(text) # 4
    text = remove_stopword(text) # 5
    text = stemming(text) # 6
    return text

In [8]:
df.replace(-1, 0, inplace=True)

In [9]:
X = df['Tweet'].apply(preprocess)

In [10]:
# Assuming the CSV file has a column named 'text' that contains the text of each document
corpus = X.tolist()
sentiments = df['sentimen'].values

In [11]:
max_features = 1000
max_len=50
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(X.values)
X = tokenizer.texts_to_sequences(X.values)
X = pad_sequences(X, padding='post', maxlen=max_len)
X.shape

In [12]:
sentiment_encode = {-1 : 0, 0 : 1, 1 : 2}
y = df['sentimen'].map(sentiment_encode).values
print(df['sentimen'])
print(y)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=1)
y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)
y_val = to_categorical(y_val, 3)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [14]:
# Convert sentiments to binary labels
binary_labels = np.array([1 if x == 1 else 0 for x in sentiments])

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the vectorizer on the corpus
tfidf_matrix = vectorizer.fit_transform(corpus)

# Get the feature names (words)
feature_names = vectorizer.get_feature_names_out()

In [15]:
# Print TF-IDF matrix and feature names
print("TF-IDF Matrix:")
print(tfidf_matrix.toarray())
print("\nFeature Names:")
print(feature_names)

Fast Text

In [28]:
# Convert the TF-IDF matrix to a dense array
dense_tfidf_matrix = tfidf_matrix.toarray()

# Initialize a FastText model
model_ft = FastText(vector_size=100, window=5, min_count=1)

# Build vocabulary and train the FastText model
model_ft.build_vocab(corpus_iterable=[text.split() for text in corpus])
model_ft.train(corpus_iterable=[text.split() for text in corpus], total_examples=len(corpus), epochs=10)

# Initialize an empty matrix to store word embeddings
word_embeddings = np.zeros((len(feature_names), model_ft.vector_size))

# Map each word to its FastText vector
for idx, word in enumerate(feature_names):
    if word in model_ft.wv:
        word_embeddings[idx] = model_ft.wv[word]

word_embed_ft = model_ft.wv["dan"]
print(word_embed_ft)

# Print word embeddings for the first few words
# print("Word Embeddings:")
# print(word_embeddings[:10])  # Print embeddings for the first 10 words

Glove

In [33]:
# Load the GloVe word vectors
glove_file = '../data/vectors.txt'
word_vectors = KeyedVectors.load_word2vec_format(glove_file, binary=False, no_header=True)

# Initialize an empty matrix to store word embeddings
word_embeddings = np.zeros((len(feature_names), word_vectors.vector_size))

# Map each word to its GloVe vector
for idx, word in enumerate(feature_names):
    if word in word_vectors:
        word_embeddings[idx] = word_vectors[word]
        
word_embed_glove = word_vectors.word_vec('dan')
print(word_embed_glove)

# Print word embeddings for the first few words
# print("Word Embeddings:")
# print(word_embeddings[:10])  # Print embeddings for the first 10 words

Word2vec- Cbow

In [31]:
# Preprocess the corpus for Word2Vec
tokenized_corpus = [text.split() for text in corpus]

# Initialize a Word2Vec CBOW model
model_w2v_cbow = Word2Vec(sentences=tokenized_corpus, vector_size=100, window=5, sg=0, min_count=1, workers=4)

# Train the Word2Vec CBOW model
model_w2v_cbow.train(tokenized_corpus, total_examples=len(tokenized_corpus), epochs=10)

# Initialize an empty matrix to store word embeddings
word_embeddings = np.zeros((len(feature_names), model_w2v_cbow.vector_size))

# Map each word to its Word2Vec CBOW vector
for idx, word in enumerate(feature_names):
    if word in model_w2v_cbow.wv:
        word_embeddings[idx] = model_w2v_cbow.wv[word]

word_embed_cbow = model_w2v_cbow.wv.most_similar('dan')
print(word_embed_cbow)

# Print word embeddings for the first few words
# print("Word Embeddings:")
# print(word_embeddings[:10])  # Print embeddings for the first 10 words

In [32]:
# Preprocess the corpus for Word2Vec
tokenized_corpus = [text.split() for text in corpus]

# Initialize a Word2Vec Skip-gram model
model_w2v_sg = Word2Vec(sentences=tokenized_corpus, vector_size=100, window=5, sg=1, min_count=1, workers=4)

# Train the Word2Vec Skip-gram model
model_w2v_sg.train(tokenized_corpus, total_examples=len(tokenized_corpus), epochs=10)

# Initialize an empty matrix to store word embeddings
word_embeddings = np.zeros((len(feature_names), model_w2v_sg.vector_size))

# Map each word to its Word2Vec Skip-gram vector
for idx, word in enumerate(feature_names):
    if word in model_w2v_sg.wv:
        word_embeddings[idx] = model_w2v_sg.wv[word]
        
word_embed_skipgram = model_w2v_sg.wv.most_similar('dan')
print(word_embed_skipgram)

# Print word embeddings for the first few words
# print("Word Embeddings:")
# print(word_embeddings[:10])  # Print embeddings for the first 10 words